In [19]:
# Dependencies

import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
import spotipy.util as util
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
import sys
import xgboost as xgb
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,roc_curve, auc,roc_auc_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
warnings.filterwarnings("ignore")

In [20]:
# Authorization
util.prompt_for_user_token(username='ENTER YOUR ID',scope = 'playlist-modify-public',client_id='ENTER YOUR ID',client_secret='a7b711f88a604200b660d2a47930133d',redirect_uri='https://example.com/callback/')
client_credentials_manager = SpotifyClientCredentials(client_id='ENTER YOUR ID', client_secret='ENTER YOUR ID') 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 


In [21]:
def make_table (uri):
    uri = uri
    username = uri.split(':')[2]
    playlist_id = uri.split(':')[4]
    results = sp.user_playlist(username, playlist_id)
    all_data = list(results.values())
    tracks_play = [list(x.values()) for x in list(all_data[12]["items"])]
    uri_ = [x[4]['uri'].split(':')[2] for x in tracks_play]
    data = pd.DataFrame(sp.audio_features(uri_))
    data['song_name'] = [x[4]['name'] for x in tracks_play]
    #data['popularity'] = [x[4]['popularity'] for x in tracks_play]
    #data['explicit'] = [0 if x[4]['explicit']==False else 1 for x in tracks_play]
    data.drop(['type','type','analysis_url','duration_ms','mode','key','track_href','id','time_signature',
               'instrumentalness','loudness','liveness'],axis=1,inplace=True)
    data['tempo'] = scaler.fit_transform(np.array(data['tempo']).reshape(-1, 1))
    data.set_index('song_name',inplace=True)
    return data

In [22]:
data = make_table('spotify:user:22jsfsrf3rxedkj3meohyummy:playlist:7kDhxtWngRnyWwNlTnkwVh')

In [23]:
data.head(2)

,acousticness,danceability,energy,speechiness,tempo,uri,valence
song_name,,,,,,,
Bloom,0.047400,0.471,0.906,0.0476,0.718170,spotify:track:1fPXCSSpwvw9BRL5i5wHlw,0.914
Your Hate Is Adorable,0.000947,0.576,0.634,0.0312,0.685516,spotify:track:0tqvhmJQhaWyWtzVZOIVg6,0.528


In [24]:
aco = data['acousticness'].mean()
dan = data['danceability'].mean()
ene = data['energy'].mean()
spe = data['speechiness'].mean()
tem = data['tempo'].mean()
val = data['valence'].mean()
means = [aco,dan,ene,spe,tem,val]

In [25]:
# Getting data from all fresh find playlists
fresh1 = make_table('spotify:user:spotify:playlist:37i9dQZF1DWWjGdmeTyeJ6')
fresh2 = make_table('spotify:user:spotify:playlist:37i9dQZF1DWUFAJPVM3HTX')
fresh3 = make_table('spotify:user:spotify:playlist:37i9dQZF1DWW4igXXl2Qkp')
fresh4 = make_table('spotify:user:spotify:playlist:37i9dQZF1DX3u9TSHqpdJC')
fresh5 = make_table('spotify:user:spotify:playlist:37i9dQZF1DX6bBjHfdRnza')
fresh6 = make_table('spotify:user:spotify:playlist:37i9dQZF1DWT0upuUFtT7o')
fresh7 = make_table('spotify:user:spotify:playlist:37i9dQZF1DX78toxP7mOaJ')
fresh = pd.concat([fresh1,fresh2,fresh3,fresh4,fresh5,fresh6,fresh7])

In [26]:
columns = [x for x in fresh.columns if x != 'uri']

In [27]:
for feature,mean in zip(columns,means):
    fresh[feature] = abs(fresh[feature] - mean)

In [57]:
fresh['score'] = fresh['acousticness'] + fresh['acousticness'] + fresh['danceability'] + fresh['energy'] + fresh['speechiness'] + fresh['tempo'] + fresh['valence']
fresh['score'] = 1 - scaler.fit_transform(np.array(fresh['score']).reshape(-1,1))
ranked = fresh.sort_values(by=['score'],ascending=False)
ranked = ranked[~ranked.index.duplicated(keep='first')]

(171, 8)

In [58]:
tracks = list(ranked['uri'])

In [59]:
def make_new_playlist(username,playlist_name):
    playlists_ = []
    token = util.prompt_for_user_token(username=username,scope='playlist-modify-public',client_id='ENTER YOUR ID',client_secret='ENTER YOUR ID',redirect_uri='https://example.com/callback/')
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        playlists = sp.user_playlist_create(username, playlist_name)
        names = sp.user_playlists(username)
        for playlist in names['items']:ENTER YOUR ID
            playlists_.append(playlist['id'])
        return playlists_[0]
        
    else:
        print("Can't get token for", username)

def add_tracks(username, playlist_id, tracks):
    token = util.prompt_for_user_token(username=username,scope='playlist-modify-public',client_id='ENTER YOUR ID',client_secret='ENTER YOUR ID',redirect_uri='https://example.com/callback/')
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        sp.user_playlist_add_tracks(username, playlist_id, tracks)
    
    

In [60]:
add_tracks('ENTER ID',make_new_playlist('ENTER ID','test5'),tracks[:20])